# 如何处理速率限制

:::info 先决条件

本指南假定您熟悉以下概念：
- [Chat models](/docs/concepts/chat_models)
- [LLMs](/docs/concepts/text_llms)
:::


您可能会发现自己处于因发出过多请求而被模型提供商 API 阻止速率限制的情况下。

例如，如果您正在运行许多并行查询来在测试数据集上对聊天模型进行基准测试，则可能会发生这种情况。

如果您遇到这种情况，可以使用速率限制器来帮助匹配您发送请求的速率与 API 允许的速率。

:::info 需要 ``langchain-core >= 0.2.24``

此功能已在 ``langchain-core == 0.2.24`` 中添加。请确保您的包已更新。
:::

## 初始化速率限制器

Langchain 自带了一个内存中的速率限制器。该速率限制器是线程安全的，可以在同一进程中的多个线程之间共享。

提供的速率限制器只能限制每单位时间内的请求数量。如果你还需要根据请求的大小进行限制，它将无法提供帮助。

In [4]:
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.1,  # <-- Super slow! We can only make a request once every 10 seconds!!
    check_every_n_seconds=0.1,  # Wake up every 100 ms to check whether allowed to make a request,
    max_bucket_size=10,  # Controls the maximum burst size.
)

## 选择一个模型

选择任意模型，并通过 `rate_limiter` 属性将其传递给该模型。

In [2]:
import os
import time
from getpass import getpass

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass()


from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model_name="claude-3-opus-20240229", rate_limiter=rate_limiter)

让我们确认一下速率限制器是否有效。我们应该每 10 秒只能调用模型一次。

In [3]:
for _ in range(5):
    tic = time.time()
    model.invoke("hello")
    toc = time.time()
    print(toc - tic)

11.599073648452759
10.7502121925354
10.244257926940918
8.83088755607605
11.645203590393066
